# Кластеризация новостей с помощью алгоритма KMeans

Для представления текста в виде вектора используется tf-idf с n-gram, для нормализации слов используется Yandex MyStem, 

# Модель новости

In [1]:
import json

class News:
    def __init__(self, id, date, annotation, title, content, url):
        self.id = id
        self.date = date
        self.annotation = annotation
        self.title = title
        self.content = content
        self.url = url
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

# Загружаем новости для кластеризации

In [2]:
news = []
with open('/data/kasandra/news-1000.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

list(map(lambda w: w.title, news[:5]))

['На Алтае заработала первая в РФ система контроля космического пространства',
 'ЛНР: Действия украинских диверсантов — одна из версий покушения на Плотницкого',
 'Bild не публикует результаты России в медальном зачёте Олимпиады',
 'Глава ЛНР связал покушение на свою жизнь с украинскими спецслужбами',
 'Мэра города в США арестовали за попытку расплатиться наркотиками за секс с мужчинами']

# Загружаем стоп слова

In [3]:
stopwords = []

with open('../main/python/res/stopwords.txt', mode="r", encoding="utf8") as file:
    for line in file:
        stopwords.append(line.replace('\n', ''))

# Подключаем Yandex MyStem для нормализации текста

In [4]:
from pymystem3 import Mystem
import re

mystem = Mystem(mystem_bin='/data/mystem/mystem',
                entire_input=False)

# Оставляем только русские слова
r = re.compile('^[А-ЯЙа-яй]*$')

def lemmatize(text):
    return list(filter(r.match, mystem.lemmatize(text)))

## Преобразуем текст в вектор с помощью TfidfVectorizer

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf=True,
                                   tokenizer=lemmatize,
                                   stop_words=stopwords,
                                   ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(map(lambda n: n.content, news))
idf = tfidf_vectorizer.idf_

Слова использующиеся в векторе

In [6]:
tfidf_vectorizer.get_feature_names()[:1000]

['аба',
 'аба ранее',
 'аба ранее террорист',
 'аба сообщаться',
 'аба сообщаться погибший',
 'абатов',
 'абатов примерно',
 'абатов примерно километр',
 'аббас',
 'аббас побережье',
 'аббас побережье персидский',
 'абдалла',
 'абдалла бин',
 'абдалла бин мусаид',
 'абдалла салех',
 'абдалла салех сообщать',
 'абдеслам',
 'абдеслам задерживать',
 'абдеслам задерживать март',
 'абдеслам занимать',
 'абдеслам занимать должность',
 'абдеслам получать',
 'абдеслам получать власть',
 'абдувахоб',
 'абдувахоб маджидов',
 'абдувахоб маджидов руслан',
 'абдул',
 'абдул рашид',
 'абдул рашид садулаев',
 'абдулкарим',
 'абдулкарим арестовывать',
 'абдулкарим арестовывать полиция',
 'абдулович',
 'абдулович искандер',
 'абдулович искандер новость',
 'абдулович искандер писатель',
 'абдулрашид',
 'абдулрашид садулаев',
 'абдулрашид садулаев анзор',
 'абдулрашид садулаев высокий',
 'абдурашид',
 'абдурашид виталий',
 'абдурашид виталий дунайцев',
 'абдурашид владимир',
 'абдурашид владимир никитин'

# Выполняем кластеризацию с помощью алгоритма K-Means

In [7]:
from sklearn.cluster import KMeans

num_clusters = 200

km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)

clusters = km.cluster_centers_
labels = km.labels_
labels[:100]

array([ 68,  36,  36,  36, 134,  17, 110, 117,  36, 143,  12,  30,  36,
        59, 124, 138, 109,  36,  22,  91, 153, 192, 178,  36, 142,  36,
       110,  43,  45,  68,  36, 105, 117,  16, 131,  92, 117,  38, 188,
        43,  46, 107,  36, 117,  72,  36,  13, 117,  36,  74,  36,  36,
        51,  93, 189,  36, 174,  61,  17,  36,  61,  38, 130,  36,  54,
       160, 174,  55, 117,  93,  66,  66, 159,  84,  43, 129,  36,  61,
       149, 114,  92,   7,  72,  61,  17, 190,  72,  38,  17,  75,  36,
        66,  36,  93, 157,  78,  14, 185,  74,  92], dtype=int32)

# Назначаем кластер новости

In [8]:
import numpy as np

zipNews = zip(news, labels)

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

newsLabels = list(map(assign_label_to_news, zipNews))
list(map(lambda n: [n.title, n.label], newsLabels))[:10]

[['На Алтае заработала первая в РФ система контроля космического пространства',
  68],
 ['ЛНР: Действия украинских диверсантов — одна из версий покушения на Плотницкого',
  36],
 ['Bild не публикует результаты России в медальном зачёте Олимпиады', 36],
 ['Глава ЛНР связал покушение на свою жизнь с украинскими спецслужбами', 36],
 ['Мэра города в США арестовали за попытку расплатиться наркотиками за секс с мужчинами',
  134],
 ['Народная милиция ЛНР назвала причастных к покушению на Плотницкого', 17],
 ['Во время церемонии открытия ОИ-2016 был ограблен Дом болельщиков сборной России',
  110],
 ['В Глазго, предположительно из-за взрыва, обрушилась часть здания', 117],
 ['Председатель Совмина ЛНР: Состояние Плотницкого после покушения стабильное',
  36],
 ['Трамп и Крым', 143]]

# Группируем новости по кластерам

In [9]:
from itertools import groupby

newsLabels = sorted(newsLabels, key=lambda n: n.label)

groupedNews = {}
for key, group in groupby(newsLabels, lambda n: n.label):
    groupedNews[str(key)] = list(map(lambda n: n.__dict__, group))

jsonData = json.dumps(groupedNews, ensure_ascii=False, sort_keys=True, indent=4, separators=(',', ': '))

# Информация о кластерах

In [10]:
for label, group in groupby(newsLabels, lambda n: n.label):
    groupList = list(group)
    print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
    for gr in groupList:
        print("\t" + gr.title)

Cluster: 0, count news: 1, titles:
	День в истории: 3 августа
Cluster: 1, count news: 3, titles:
	Каноист Крайтор не явился на контрольный заезд для допуска на Олимпиаду
	ICF допустила российского гребца-каноиста Андрея Крайтора на Олимпиаду
	ICF пересмотрела свое решение и допустила каноиста Крайтора на Олимпиаду
Cluster: 2, count news: 9, titles:
	Барак Обама: Террористы-одиночки и малые группы могут совершить теракты в США
	Бюрократия терроризма: бывший боевик рассказал, как организована вербовка в ИГ
	Италия расследует причастность ИГ к управлению потоком беженцев в страну
	Новая кампания США в Ливии: борьба с ИГ или вмешательство во внутренние дела страны?
	В Вашингтоне офицер полиции арестован по подозрению в связях с ИГ
	Госдеп США опроверг данные о планах отправки сухопутных войск в Ливию
	Дональд Трамп выступил за союз России и США в борьбе с ИГ
	Во Франции задержали мужчину по делу о теракте в Ницце
	Выходец из Алжира угрожал взорвать торговый центр в немецком Бремене
Cluster

# Сохранение в файл

In [ ]:
import datetime

file_path = '/data/kasandra/results/tfidf_kmeans_%s' % datetime.datetime.now().timestamp()

with open(file_path, mode="w+", encoding="utf8") as file:
    file.write(jsonData)
    
file_path